### Azpn : Azure VPN Free (For Students Plan)
_Premise: Azure student plan gives students a good 100 dollars starter credits yearly, we can use this quota to provide ourself with very cheap but useful VPN Server_

__Whatchu Need?__

A student email id (with .edu ending in last) and verified azure student account, If you are a student and signup to azure, It gives you 100\\$ per year until you graduate! So simply exploiting this loophole and deploying server using those 100\\$ of free creds is what Azpn do! Most people would never even use Azure much, if you don't mind using some of your credits for a free secure VPN then continue reading! :)

## Helper functions

In [ ]:
import sys
import subprocess

def execute(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    # Poll process for new output until finished
    r = ""
    while True:
        nextline = process.stdout.readline()
        if not nextline and process.poll() is not None:
            break
        r += nextline.decode("utf-8") .strip() + " "
        sys.stdout.write(str(nextline))
        sys.stdout.flush()

    output = process.communicate()[0]
    exitCode = process.returncode

    if (exitCode == 0):
        return r
    else:
        print("error?")
        
        
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
def generate_temp_password(length):
    if not isinstance(length, int) or length < 8:
        raise ValueError("temp password must have positive length")

    chars = "@ABCDEFGHJKLMNPQRSTUVWXYZ23456789abcdefghijklmnopqrstuvwxyz@"
    from os import urandom

    # original Python 2 (urandom returns str)
    # return "".join(chars[ord(c) % len(chars)] for c in urandom(length))

    # Python 3 (urandom returns bytes)
    return "".join(chars[c % len(chars)] for c in urandom(length))

def downloadFile(fname):
    stdin,stdout,stderr=ssh.exec_command("sudo cat /home/%s/%s"%(ADMIN_USERNAME, fname))
    outlines=stdout.readlines()
    resp=''.join(outlines)
    print(resp)

    with open(fname, 'w') as file:
        file.write(resp)

### Global Configs

In [ ]:
import uuid
import secrets
import string

RESOURCE_GROUP_NAME = "azpn_group"
VM_INSTANCE_NAME = "azpn_instance001"
MACHINE_NAME = "Standard_B1ls" # Cheapest, Scale
ADMIN_USERNAME = "azpn"
ADMIN_PASSWORD = generate_temp_password(48) # Unique one time password

In [ ]:
printmd("__Admin password generated, please save this somewhere or you risk losing access to your virtual machine__")
printmd("PASSWORD IS __%s__"%(ADMIN_PASSWORD))

### Start
_login azure-cli_

In [ ]:
x = execute("az login")
if("Azure for Students" not in x):
    printmd("**Wrong account? Couldn't find azure students plan or cannot login to service, try again!**")

__Get all resource regions (ie VPN Region)__

In [ ]:
locs = execute('az account list-locations --query "[].{DisplayName:displayName, Name:name}" -o table')

__Input your preferred region__

In [ ]:
def reg():
    r = input("Enter your preferred region (default-centralindia): ") or "centralindia"
    if("            " + r + " " not in locs):
        print("Invalid/Bad region, enter again.")
        reg()
    else:
        return r

REGION = reg()
printmd("_Selected region: %s_"%(REGION))

_Create resource group for VM and other resources_

In [ ]:
x = execute("az group create --name %s --location %s --tags USE=AZPN -o table"%(RESOURCE_GROUP_NAME, REGION))

In [ ]:
c = """az vm create
    --resource-group %s
    --name %s
    --size %s
    --image Canonical:0001-com-ubuntu-server-focal:20_04-lts:latest
    --public-ip-sku Basic
    --public-ip-address-allocation dynamic
    --admin-username %s
    --admin-password %s
    --storage-sku Standard_LRS
    --location %s
    --data-disk-delete-option Delete
    --nic-delete-option Delete
    --data-disk-delete-option Delete
    -o json""" % (RESOURCE_GROUP_NAME, VM_INSTANCE_NAME, MACHINE_NAME,ADMIN_USERNAME, ADMIN_PASSWORD, REGION)
print(c)
x = execute(c.replace("\n",""))

In [ ]:
import json
vm_data = json.loads(x)
if('publicIpAddress' not in vm_data):
    printmd("__Cannot find public ip address of vm, creation problem perhaps?__")

In [ ]:
print(vm_data)

In [ ]:
SSH_IP = vm_data["publicIpAddress"]
# SSH_IP = "20.207.202.196"
# ADMIN_PASSWORD = "passs"
SSH_PORT = 22

printmd("__Connecting to %s:%s__"%(SSH_IP, SSH_PORT))

import paramiko
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(SSH_IP,SSH_PORT,ADMIN_USERNAME,ADMIN_PASSWORD)

printmd("__Connected! Upgrading packages.. (might take a while)__")

In [ ]:
stdin,stdout,stderr=ssh.exec_command("sudo apt-get update -y && sudo apt-get upgrade -y")
outlines=stdout.readlines()
resp=''.join(outlines)
print(resp)

_Download openvpn-install.sh file_

In [ ]:
_, _, _ = ssh.exec_command("curl -O https://raw.githubusercontent.com/angristan/openvpn-install/master/openvpn-install.sh && chmod +x openvpn-install.sh")

In [ ]:
stdin,stdout,stderr=ssh.exec_command("sudo AUTO_INSTALL=y PORT_CHOICE=2 PORT=443 PROTOCOL_CHOICE=2 ./openvpn-install.sh")
outlines=stdout.readlines()
opnout=''.join(outlines)
print(opnout)

In [ ]:
if not "The configuration file has been written to" in opnout:
    printmd("__Error in installation or whaat?__")

printmd("__Downloading client.ovpn for connection!__")
downloadFile("client.ovpn")    
printmd("__Downloaded! Enjoy!! :P__")

### Lastly open 443 port for our VPN

In [ ]:
x = execute("az vm open-port --resource-group %s --name %s --port 443"%(RESOURCE_GROUP_NAME, VM_INSTANCE_NAME))

## SUCCESS

If you reached here, it mean's you have successfully deployed a working VPN on your azure server! Now what? Simply Install latest openvpn client and connect to profile _client.ovpn_(in same directory)!

_Also, if you ever want to save your creds by deleting everything azpn has done until now, then execute commands in next catalog (3), namely azpn_remove_all, It will undo any changes made by this script!_

# Bonus
What if you want to share your created VPN with multiple users? Well don't worry, this bonus section is for those people wanting to generate more than one profiles for openvpn, So as to share it with their friends without them needing to install anything!

In [ ]:
def no():
    try:
        x = int(input("How many profiles to generate?"))
        return x
    except:
        print("Invalid input, enter a number!")
        return no()
    
PROFILE_NO = no()
for i in range(PROFILE_NO):
    confName = "client{}".format(i+1)
    cmd = 'sudo MENU_OPTION="1" CLIENT="%s" PASS="1" ./openvpn-install.sh'%(confName)
    print(cmd)
    stdin,stdout,stderr=ssh.exec_command(cmd)
    outlines=stdout.readlines()
    resp=''.join(outlines)
    print(resp)
    downloadFile("%s.ovpn"%(confName))    